#Projekt_3

## Moduły

In [1]:
import os
import json
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import torch
import torch.nn as nn
import sys
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision.utils import save_image
from tqdm import tqdm
import torchvision.models as models
from google.colab import drive
import matplotlib.pyplot as plt
import numpy as np

## Upload datasetu z dysku

In [2]:
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
ROOT_PROJECT_FOLDER = '/content/drive/MyDrive/projekt_3'

##Dataset

In [4]:
class PhongDataset(Dataset):
    # Zakresy teoretyczne wektorów relatywnych:
    MAX_VECTOR_RANGE = 30.0

    SHININESS_MIN = 3.0
    SHININESS_MAX = 20.0

    def __init__(self, root_dir):
        self.root_dir = root_dir

        self.files = sorted([f for f in os.listdir(root_dir) if f.endswith('.json')])

        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

    def __len__(self):
        return len(self.files)

    @staticmethod
    def normalize_data(rel_light, rel_view, diffuse, shininess):
        norm_vector = []

        # w. Relatywne )
        # [-30, 30] -> [-1, 1]
        norm_vector.extend([x / PhongDataset.MAX_VECTOR_RANGE for x in rel_light])
        norm_vector.extend([x / PhongDataset.MAX_VECTOR_RANGE for x in rel_view])

        # [0.0, 1.0] do [-1, 1]
        norm_vector.extend([(x * 2.0) - 1.0 for x in diffuse])

        # [3.0, 20.0] do [-1, 1]
        s_01 = (shininess - PhongDataset.SHININESS_MIN) / (PhongDataset.SHININESS_MAX - PhongDataset.SHININESS_MIN)
        s_norm = (s_01 * 2.0) - 1.0
        norm_vector.append(s_norm)

        return norm_vector

    def __getitem__(self, idx):
        json_name = self.files[idx]
        json_path = os.path.join(self.root_dir, json_name)

        with open(json_path, 'r') as f:
            data = json.load(f)

        input_list = PhongDataset.normalize_data(
            rel_light=data["relative_light_vector"],
            rel_view=data["relative_view_vector"],
            diffuse=data["material_diffuse"],
            shininess=data["material_shininess"]
        )

        input_tensor = torch.tensor(input_list, dtype=torch.float32)

        # obraz
        img_name = data["file_name"]
        img_path = os.path.join(self.root_dir, img_name)

        image = Image.open(img_path).convert("RGB")
        image_tensor = self.transform(image)

        return input_tensor, image_tensor

## Model

In [26]:
import torch
import torch.nn as nn


PARAM_DIM = 10          # Wymiar parametrów IN
IMG_CHANNELS = 3        # RGB
IMG_SIZE = 128          # 128x128

class Generator(nn.Module):

    def __init__(self):
        super(Generator, self).__init__()

        self.input_dim = PARAM_DIM

        self.initial_layer = nn.Sequential(
            nn.Linear(self.input_dim, 512),
            nn.ReLU(True),

            nn.Linear(512, 512),
            nn.ReLU(True),

            nn.Linear(512, 512 * 4 * 4),
            nn.ReLU(True)
        )

        self.model = nn.Sequential(
            # Start: 512 x 4 x 4

            # 4x4 -> 8x8
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            # 8x8 -> 16x16
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            # 16x16 -> 32x32
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),

            # 32x32 -> 64x64
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(True),

            # 64x64 -> 128x128 (Output)
            nn.ConvTranspose2d(32, IMG_CHANNELS, kernel_size=4, stride=2, padding=1, bias=False),
            nn.Tanh() # Wyjście [-1, 1]
        )

    def forward(self, noise, labels):

        x = self.initial_layer(labels)

        # Formowanie obrazka
        x = x.view(-1, 512, 4, 4)
        img = self.model(x)
        return img

'''
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        # z 128x128 do 4x4
        self.image_processing = nn.Sequential(
            #  3 x 128 x 128

            # 128 -> 64
            nn.Conv2d(IMG_CHANNELS, 32, kernel_size=4, stride=2, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            # 8 -> 4
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True)
            #512, 4, 4
        )

        # po spłaszczeniu obrazu (512*4*4 = 8192 cechy) + dodatkowo 10 parametrów IN
        self.classifier = nn.Sequential(
            nn.Linear(512 * 4 * 4 + PARAM_DIM, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )

    def forward(self, img, labels):
        features = self.image_processing(img)
        features_flat = features.view(features.size(0), -1)

        concat_input = torch.cat((features_flat, labels), dim=1)

        validity = self.classifier(concat_input)
        return validity
        '''

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        # 1. Label Embedding (KLUCZOWA ZMIANA)
        # Rozszerzamy 10 parametrów do 512, żeby były "ważne" dla sieci.
        self.label_embedding = nn.Sequential(
            nn.Linear(PARAM_DIM, 512),
            nn.LeakyReLU(0.2, inplace=True)
        )

        self.image_processing = nn.Sequential(
            nn.Conv2d(IMG_CHANNELS, 32, kernel_size=4, stride=2, padding=1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.InstanceNorm2d(64, affine=True), # <--- ZMIANA
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.InstanceNorm2d(128, affine=True), # <--- ZMIANA
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.InstanceNorm2d(256, affine=True), # <--- ZMIANA
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1, bias=False),
            nn.InstanceNorm2d(512, affine=True), # <--- ZMIANA
            nn.LeakyReLU(0.2, inplace=True)
        )

        # 3. Klasyfikator
        # Wejście: 8192 (Obraz) + 512 (Parametry) = 8704
        self.classifier = nn.Sequential(
            nn.Linear(512 * 4 * 4 + 512, 512), # Zmieniony wymiar wejścia!
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.5),  # 30% szansy na wyłączenie neuronu -> zapobiega kuciu na pamięć # przy dynamic wdrozone - przeciwko overfittingu
            nn.Linear(512, 1),
            nn.Sigmoid()
        )

    def forward(self, img, labels):
        features = self.image_processing(img)
        features_flat = features.view(features.size(0), -1)

        # Tu dzieje się magia łączenia:
        label_emb = self.label_embedding(labels)
        concat_input = torch.cat((features_flat, label_emb), dim=1)

        validity = self.classifier(concat_input)
        return validity

def weights_init(m):
    classname = m.__class__.__name__
    if 'Conv' in classname:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif 'BatchNorm' in classname:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)


##Parametry treningu

In [11]:
LEARNING_RATE_GEN = 0.0002
LEARNING_RATE_DISC = 0.00002
BATCH_SIZE = 16
NUM_EPOCHS = 400
L1_LAMBDA = 30#40

LOAD_MODEL = False
START_EPOCH = 0

DISC_WARMUP_EPOCHS = 0

dataset_dir = ROOT_PROJECT_FOLDER + "/dataset"
checkpoint_dir = ROOT_PROJECT_FOLDER + "/checkpoints_v6"
evaluation_dir = ROOT_PROJECT_FOLDER + "/evaluation_samples_v6" #v3 - dynamic

## Trening

In [20]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


# Definicje globalne (potrzebne do normalizacji VGG)
VGG_WEIGHT = 0.6
WARMUP_EPOCHS = 89
VGG_MEAN = torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1).to(DEVICE)
VGG_STD = torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1).to(DEVICE)


def setup_vgg_model(device):
    """
    Inicjalizuje model VGG16 i obcina go do warstw cech (Perceptual Loss).
    """
    vgg = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1).features.to(device)

    # Warstwy cech do porównywania (relu1_2, relu2_2, relu3_3)
    feature_layers = [4, 9, 16]
    blocks = []

    current_layer = 0
    for i in feature_layers:
        block = nn.Sequential(*vgg[current_layer:i]).eval()
        for p in block.parameters():
            p.requires_grad = False  # Zamrażamy wagi
        blocks.append(block)
        current_layer = i

    return nn.ModuleList(blocks).to(device)

def calculate_vgg_loss(vgg_model, fake_img, real_img):

    fake = nn.functional.interpolate(fake_img, size=(224, 224), mode='bilinear', align_corners=False)
    real = nn.functional.interpolate(real_img, size=(224, 224), mode='bilinear', align_corners=False)

    fake = (fake - VGG_MEAN) / VGG_STD
    real = (real - VGG_MEAN) / VGG_STD


    loss = 0.0
    for block in vgg_model:
        fake = block(fake)
        real = block(real)

        loss += torch.nn.functional.l1_loss(fake, real)

    return loss


os.makedirs(checkpoint_dir, exist_ok=True)
os.makedirs(evaluation_dir, exist_ok=True)



def calculate_masked_loss(fake, real, lambda_val):

    l1_diff = torch.abs(fake - real)


    mask = (real > -0.98).float()
    mask_pct = mask.mean().item() # Ile % obrazka to kula

    weights = 1.0 + (mask * 19.0)

    loss = (l1_diff * weights).mean() * lambda_val

    return loss, l1_diff.mean(), mask_pct

def save_checkpoint(model, optimizer, filename="checkpoint.pth"):
    print(f"=> Zapisywanie checkpointu do {filename}")
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)

def load_checkpoint(checkpoint_file, model, optimizer, lr):
    print(f"=> Wczytywanie checkpointu {checkpoint_file}")
    checkpoint = torch.load(checkpoint_file, map_location=DEVICE)
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

    for param_group in optimizer.param_groups:
        param_group["lr"] = lr


def get_loaders(root_dir, batch_size):
    # Podział: Train (2200), Val (200), Test (600)
    # Deterministyczny dzięki manual_seed(42)

    dataset = PhongDataset(root_dir)

    total_len = len(dataset)
    test_len = 600
    val_len = 200
    train_len = total_len - test_len - val_len # (2200)

    generator = torch.Generator().manual_seed(42)

    train_ds, val_ds, test_ds = random_split(
        dataset,
        [train_len, val_len, test_len],
        generator=generator
    )

    print(f"Podział danych: Train={len(train_ds)}, Val={len(val_ds)}, Test={len(test_ds)}")

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True) #pod cpu
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader

def check_accuracy(val_loader, gen, disc, device, epoch, l1_loss_fn, bce_loss_fn, folder=evaluation_dir):
    gen.eval()
    disc.eval()

    # Akumulatory błędów do statystyki
    total_raw_l1 = 0.0
    total_masked_l1 = 0.0
    total_g_loss = 0.0 # Loss generatora (tylko część GAN + L1, bez VGG w eval)
    total_d_loss = 0.0

    total_fake_score = 0.0 # Średnia predykcja dla Fake
    total_real_score = 0.0 # Średnia predykcja dla Real

    correct_real = 0
    correct_fake = 0
    total_samples = 0
    num_batches = 0

    print(f"\n=== RAPORT VALIDATION (EPOCH {epoch}) ===")

    with torch.no_grad():
        for i, (inputs, targets) in enumerate(val_loader):
            inputs = inputs.to(device)
            targets = targets.to(device)
            batch_size = inputs.shape[0]
            fake_images = gen(None, inputs)


            d_real = disc(targets, inputs)
            loss_d_real = bce_loss_fn(d_real, torch.ones_like(d_real)) # Bez label smoothing w eval

            # Fake
            d_fake = disc(fake_images, inputs)
            loss_d_fake = bce_loss_fn(d_fake, torch.zeros_like(d_fake))

            # Loss D
            d_loss = (loss_d_real + loss_d_fake) / 2
            total_d_loss += d_loss.item()

            # Statystyki D
            total_real_score += d_real.mean().item()
            total_fake_score += d_fake.mean().item()

            # Accuracy
            correct_real += (d_real > 0.5).sum().item()
            correct_fake += (d_fake < 0.5).sum().item()
            total_samples += batch_size


            # Loss G
            loss_g_gan = bce_loss_fn(d_fake, torch.ones_like(d_fake))

            # Loss L1 (Masked)
            masked_l1, raw_l1, _ = calculate_masked_loss(fake_images, targets, L1_LAMBDA)

            # Całkowity Loss G (dla statystyk, bez VGG)
            g_loss = loss_g_gan + masked_l1

            total_g_loss += g_loss.item()
            total_masked_l1 += masked_l1.item()
            total_raw_l1 += raw_l1.item()

            num_batches += 1

            # --- ZAPIS OBRAZKA (Pierwszy batch) ---
            if i == 0:

                debug_mask = (targets > -0.98).float()
                img_grid = torch.cat((targets[:16], fake_images[:16], debug_mask[:16]), dim=0)
                save_image(img_grid * 0.5 + 0.5, f"{folder}/epoch_{epoch}.png")


    avg_raw_l1 = total_raw_l1 / num_batches
    avg_masked_l1 = total_masked_l1 / num_batches
    avg_g_loss = total_g_loss / num_batches
    avg_d_loss = total_d_loss / num_batches

    avg_real_score = total_real_score / num_batches
    avg_fake_score = total_fake_score / num_batches

    # Dokładność (Accuracy)
    acc_real = correct_real / total_samples
    acc_fake = correct_fake / total_samples

    print("-" * 75)
    print(f"LOSSES  => G: {avg_g_loss:.4f} | D: {avg_d_loss:.4f}")
    print(f"L1      => Raw: {avg_raw_l1:.4f} | Weighted: {avg_masked_l1:.4f}")
    print(f"SCORES  => Real(D): {avg_real_score:.4f} | Fake(D): {avg_fake_score:.4f}")
    print(f"ACC     => Real: {acc_real:.2%} | Fake: {acc_fake:.2%}")
    print("=" * 75)

    gen.train()
    disc.train()

    return (avg_g_loss, avg_d_loss, avg_raw_l1, avg_masked_l1, avg_real_score, avg_fake_score , acc_real, acc_fake)

ACC_TARGET_MIN = 0.70
ACC_TARGET_MAX = 0.85
MAX_D_REPEATS = 4
MAX_G_REPEATS = 1

def train_dynamic_fn(disc, gen, loader, opt_disc, opt_gen, l1_loss, bce, epoch, vgg_criterion):
    loop = tqdm(loader, leave=True)

    # Statystyki globalne epoki
    d_total_samples = 0
    d_real_correct = 0
    d_fake_correct = 0


    total_d_boosts = 0  # Ile razy D musiał trenować więcej niż 1 raz
    total_g_boosts = 0  # Ile razy G musiał trenować 2 razy
    total_d_steps = 0   # Łączna liczba kroków D
    total_g_steps = 0   # Łączna liczba kroków G

    for idx, (inputs, real_images) in enumerate(loop):
        inputs = inputs.to(DEVICE)
        real_images = real_images.to(DEVICE)
        batch_size = inputs.shape[0]

        d_loops = 0

        while d_loops < MAX_D_REPEATS:
            # Generujemy fake (bez grafu dla G)
            with torch.no_grad():
                fake_images = gen(None, inputs)

            # --- REAL ---
            d_real = disc(real_images, inputs)
            loss_d_real = bce(d_real, torch.ones_like(d_real) * 0.9) # Smooth labels

            # --- FAKE ---
            d_fake = disc(fake_images.detach(), inputs)
            loss_d_fake = bce(d_fake, torch.zeros_like(d_fake))

            loss_d = (loss_d_real + loss_d_fake) / 2

            # Accuracy składowe
            acc_real = (d_real > 0.5).float().mean().item()
            acc_fake = (d_fake < 0.5).float().mean().item()
            current_acc = (acc_real + acc_fake) / 2



            is_unbalanced = abs(acc_real - acc_fake) > 0.3
            is_blind_to_real = acc_real < 0.45
            is_weak_overall = current_acc < ACC_TARGET_MAX

            # Decyzja o treningu D
            should_train_d = (is_weak_overall or is_blind_to_real or is_unbalanced)

            if current_acc > 0.95: should_train_d = False

            if should_train_d:
                disc.zero_grad()
                loss_d.backward()
                opt_disc.step()
                d_loops += 1
                total_d_steps += 1
            else:
                break # D jest wystarczająco dobry/zbalansowany

            # Wyjście awaryjne (jeśli już jest super)
            if current_acc > ACC_TARGET_MIN and not is_blind_to_real:
                break

        # Statystyki
        d_real_correct += (d_real > 0.5).sum().item()
        d_fake_correct += (d_fake < 0.5).sum().item()
        d_total_samples += batch_size

        if d_loops > 1: total_d_boosts += 1


        # Jeśli D jest za mądry, G trenuje 2 razy
        g_repeats = MAX_G_REPEATS if current_acc >= ACC_TARGET_MAX else 1
        if g_repeats > 1: total_g_boosts += 1

        for i in range(g_repeats):
            noise = torch.randn(batch_size, LATENT_DIM, device=device)
            fake_for_g = gen(noise, inputs)
            d_fake_g = disc(fake_for_g, inputs)

            loss_g_gan = bce(d_fake_g, torch.ones_like(d_fake_g))


            loss_g_l1, raw_l1, _ = calculate_masked_loss(fake_for_g, real_images, L1_LAMBDA)

            g_loss = loss_g_gan + loss_g_l1

            gen.zero_grad()
            g_loss.backward()
            opt_gen.step()
            total_g_steps += 1


        status_msg = "OK"
        if d_loops > 1: status_msg = f"D+{d_loops-1}"
        if g_repeats > 1: status_msg = f"G+{g_repeats-1}"

        loop.set_postfix(
            St=status_msg,
            D=f"{loss_d.item():.3f}",
            G=f"{g_loss.item():.2f}",
            L1=f"{raw_l1.item():.4f}",
            L_W=f"{loss_g_l1.item():.2f}",
            AccR=f"{acc_real:.2f}",
            AccF=f"{acc_fake:.2f}"
        )

    final_acc_real = d_real_correct / d_total_samples if d_total_samples > 0 else 0
    final_acc_fake = d_fake_correct / d_total_samples if d_total_samples > 0 else 0

    print(f"\n[Dynamic Stats] D_Boosts: {total_d_boosts} | G_Boosts: {total_g_boosts} || Total Steps: D={total_d_steps}, G={total_g_steps}")
    print(f"[Epoch Stats]   Acc Real: {final_acc_real:.2%} | Acc Fake: {final_acc_fake:.2%}")

def train_fn(disc, gen, loader, opt_disc, opt_gen, l1_loss, bce, epoch, vgg_criterion):
    loop = tqdm(loader, leave=True)

    d_real_correct = 0
    d_fake_correct = 0
    d_total = 0

    for idx, (inputs, real_images) in enumerate(loop):
        inputs = inputs.to(DEVICE)
        real_images = real_images.to(DEVICE)
        batch_size = inputs.shape[0]


        if epoch < WARMUP_EPOCHS:
            fake_images = gen(None, inputs)

            loss_g_l1, raw_l1, mask_pct = calculate_masked_loss(fake_images, real_images, L1_LAMBDA)

            # Tylko L1
            g_loss = loss_g_l1

            gen.zero_grad()
            g_loss.backward()
            opt_gen.step()

            loop.set_postfix(Mode="WARMUP_L1", L1=f"{raw_l1:.4f}", TotalG=f"{g_loss.item():.2f}")


        else:

            freeze_gen = epoch < (START_EPOCH + DISC_WARMUP_EPOCHS)
            mode_name = "REHAB_DISC" if freeze_gen else "GAN_TRAIN"


            with torch.set_grad_enabled(not freeze_gen):
                fake_images = gen(None, inputs)

            # Czyste obrazy (BEZ SZUMU)
            # Real
            d_real = disc(real_images, inputs)
            loss_d_real = bce(d_real, torch.ones_like(d_real) * 0.9) # Label smoothing
            d_real_correct += (d_real > 0.5).sum().item()

            # Fake (detach, bo uczymy tylko D)
            d_fake = disc(fake_images.detach(), inputs)
            loss_d_fake = bce(d_fake, torch.zeros_like(d_fake))
            d_fake_correct += (d_fake < 0.5).sum().item()

            loss_d = (loss_d_real + loss_d_fake) / 2

            disc.zero_grad()
            loss_d.backward()
            opt_disc.step()

            if not freeze_gen:
                # Trenujemy G tylko w trybie GAN_TRAIN

                # Oszukujemy Dyskryminatora
                d_fake_for_gen = disc(fake_images, inputs)
                loss_g_gan = bce(d_fake_for_gen, torch.ones_like(d_fake_for_gen))

                # Pilnujemy geometrii (Masked L1)
                loss_g_l1, raw_l1, mask_pct = calculate_masked_loss(fake_images, real_images, L1_LAMBDA)

                # Suma celów
                g_loss = loss_g_gan + loss_g_l1

                gen.zero_grad()
                g_loss.backward()
                opt_gen.step()

                # Wartości do logów
                log_g_loss = g_loss.item()
                log_l1 = raw_l1.item()
            else:
                # W trybie REHAB tylko podglądamy L1, ale nie aktualizujemy wag G
                with torch.no_grad():
                     _, log_l1, _ = calculate_masked_loss(fake_images, real_images, L1_LAMBDA)
                log_g_loss = 0.0

            # Logowanie
            d_total += batch_size
            acc_real = d_real_correct / d_total if d_total > 0 else 0
            acc_fake = d_fake_correct / d_total if d_total > 0 else 0

            loop.set_postfix(
                Mode=mode_name,
                D=f"{loss_d.item():.4f}",
                G=f"{log_g_loss:.2f}",
                L1=f"{log_l1:.4f}",
                AccR=f"{acc_real:.2f}",
                AccF=f"{acc_fake:.2f}"
            )

In [27]:
print(f"Urządzenie: {DEVICE}")
print(f"Lambda L1: {L1_LAMBDA}")

gen = Generator().to(DEVICE)
disc = Discriminator().to(DEVICE)

gen.apply(weights_init)
disc.apply(weights_init)

opt_gen = optim.Adam(gen.parameters(), lr=LEARNING_RATE_GEN, betas=(0.5, 0.999))
opt_disc = optim.Adam(disc.parameters(), lr=LEARNING_RATE_DISC, betas=(0.5, 0.999)) # disc uczy się 4 razy wolniej

vgg_criterion = setup_vgg_model(DEVICE)
print("Zainicjalizowano Generator VGG Loss (VGG16).")

bce = nn.BCELoss()
l1_loss = nn.L1Loss()

train_loader, val_loader, test_loader = get_loaders(dataset_dir, BATCH_SIZE)


Urządzenie: cuda
Lambda L1: 30
Zainicjalizowano Generator VGG Loss (VGG16).
Podział danych: Train=2200, Val=200, Test=600


In [ ]:
if LOAD_MODEL:
    current_lr_gen = LEARNING_RATE_GEN
    current_lr_disc = LEARNING_RATE_DISC
    load_checkpoint(f"{checkpoint_dir}/gen_epoch_{START_EPOCH}.pth", gen, opt_gen, current_lr_gen)
    load_checkpoint(f"{checkpoint_dir}/disc_epoch_{START_EPOCH}.pth", disc, opt_disc, current_lr_disc)
    print(f"--> Wznowiono trening od epoki {START_EPOCH}. Nowe LR: G={current_lr_gen}, D={current_lr_disc}")

In [ ]:
train_info = []

os.makedirs(checkpoint_dir, exist_ok=True)
train_params = f"\n\n### PARAMETRY TRENINGU ### LEARNING_RATE_GEN={LEARNING_RATE_GEN}\nLEARNING_RATE_DISC={LEARNING_RATE_DISC}\nBATCH_SIZE={BATCH_SIZE}\nNUM_EPOCHS={NUM_EPOCHS}\nL1_LAMBDA={L1_LAMBDA}"
with open(f"{checkpoint_dir}/train_params.txt", "w") as f:
  f.write(train_params)

for epoch in range(START_EPOCH + 1, START_EPOCH + 1 + NUM_EPOCHS):
    print(f"Epoch [{epoch}/{START_EPOCH + NUM_EPOCHS}]")

    train_dynamic_fn(disc, gen, train_loader, opt_disc, opt_gen, l1_loss, bce, epoch, vgg_criterion)
    info = check_accuracy(val_loader, gen, disc, DEVICE, epoch, l1_loss,bce)

    train_info.append(info)

    if epoch == 50 and epoch == 100 and epoch == 150 and epoch == 180 and epoch >= 197:
      save_checkpoint(gen, opt_gen, filename=f"{checkpoint_dir}/gen_epoch_{epoch}.pth")
      save_checkpoint(disc, opt_disc, filename=f"{checkpoint_dir}/disc_epoch_{epoch}.pth")

Epoch [1/200]


100%|██████████| 138/138 [00:20<00:00,  6.88it/s, AccF=1.00, AccR=0.75, D=0.455, G=25.78, L1=0.1701, L_W=24.53, St=OK]



[Dynamic Stats] D_Boosts: 9 | G_Boosts: 0 || Total Steps: D=80, G=138
[Epoch Stats]   Acc Real: 76.77% | Acc Fake: 95.55%

=== RAPORT VALIDATION (EPOCH 1) ===
---------------------------------------------------------------------------
LOSSES  => G: 24.1373 | D: 0.4184
L1      => Raw: 0.1741 | Weighted: 22.9605
SCORES  => Real(D): 0.6653 | Fake(D): 0.3092
ACC     => Real: 80.50% | Fake: 100.00%
Epoch [2/200]


100%|██████████| 138/138 [00:19<00:00,  6.94it/s, AccF=1.00, AccR=1.00, D=0.323, G=30.43, L1=0.1040, L_W=28.97, St=OK]



[Dynamic Stats] D_Boosts: 2 | G_Boosts: 0 || Total Steps: D=74, G=138
[Epoch Stats]   Acc Real: 73.00% | Acc Fake: 99.18%

=== RAPORT VALIDATION (EPOCH 2) ===
---------------------------------------------------------------------------
LOSSES  => G: 19.2765 | D: 0.3795
L1      => Raw: 0.0905 | Weighted: 17.9636
SCORES  => Real(D): 0.7367 | Fake(D): 0.2692
ACC     => Real: 75.00% | Fake: 100.00%
Epoch [3/200]


...


[Dynamic Stats] D_Boosts: 0 | G_Boosts: 0 || Total Steps: D=26, G=138
[Epoch Stats]   Acc Real: 87.05% | Acc Fake: 95.86%

=== RAPORT VALIDATION (EPOCH 198) ===
---------------------------------------------------------------------------
LOSSES  => G: 18.6918 | D: 0.3498
L1      => Raw: 0.0790 | Weighted: 17.3254
SCORES  => Real(D): 0.7404 | Fake(D): 0.2775
ACC     => Real: 85.50% | Fake: 92.00%
Epoch [199/200]


100%|██████████| 138/138 [00:19<00:00,  6.94it/s, AccF=1.00, AccR=1.00, D=0.343, G=27.84, L1=0.0932, L_W=25.96, St=OK]



[Dynamic Stats] D_Boosts: 0 | G_Boosts: 0 || Total Steps: D=27, G=138
[Epoch Stats]   Acc Real: 87.32% | Acc Fake: 94.41%

=== RAPORT VALIDATION (EPOCH 199) ===
---------------------------------------------------------------------------
LOSSES  => G: 19.3546 | D: 0.3313
L1      => Raw: 0.0770 | Weighted: 17.8421
SCORES  => Real(D): 0.7266 | Fake(D): 0.2387
ACC     => Real: 85.00% | Fake: 96.00%
Epoch [200/200]


100%|██████████| 138/138 [00:19<00:00,  6.97it/s, AccF=0.88, AccR=0.88, D=0.350, G=23.39, L1=0.0859, L_W=22.04, St=OK]



[Dynamic Stats] D_Boosts: 0 | G_Boosts: 0 || Total Steps: D=16, G=138
[Epoch Stats]   Acc Real: 90.59% | Acc Fake: 93.59%

=== RAPORT VALIDATION (EPOCH 200) ===
---------------------------------------------------------------------------
LOSSES  => G: 18.6614 | D: 0.3193
L1      => Raw: 0.0787 | Weighted: 17.1406
SCORES  => Real(D): 0.7434 | Fake(D): 0.2350
ACC     => Real: 84.00% | Fake: 99.50%


## Wyniki

In [ ]:
test_folder = ROOT_PROJECT_FOLDER + "/test_v6"
result_folder = ROOT_PROJECT_FOLDER + "/result_v6"

### Wykresy

In [ ]:
def show_plt(data, save_folder=None):
  epoches_x_label = range(len(data))

  fig, axes = plt.subplots(2, 2, figsize=(10,8))
  # Plot 1 ---------------------------------------------------------------
  axes[0,0].set_title('Validation loss (g_loss i masked_l1)')
  axes[0,0].set_xlabel('Epochs')
  axes[0,0].set_ylabel('Loss')

  axes[0,0].plot(epoches_x_label, np.array([x[0] for x in data]), label='g_loss')
  axes[0,0].plot(epoches_x_label, np.array([x[3] for x in data]), label='masked_l1')

  axes[0,0].legend()
  axes[0,0].grid(True)

  # Plot 2 ---------------------------------------------------------------
  axes[0,1].set_title('Validation loss (d_loss i raw_l1)')
  axes[0,1].set_xlabel('Epochs')
  axes[0,1].set_ylabel('Loss')

  axes[0,1].plot(epoches_x_label, np.array([x[1] for x in data]), label='d_loss')
  axes[0,1].plot(epoches_x_label, np.array([x[2] for x in data]), label='raw_l1')

  axes[0,1].legend()
  axes[0,1].grid(True)

  # Plot 3 ---------------------------------------------------------------
  axes[1,0].set_title('Validation scores')
  axes[1,0].set_xlabel('Epochs')
  axes[1,0].set_ylabel('Scores')

  axes[1,0].plot(epoches_x_label, np.array([x[4] for x in data]), label='real_score')
  axes[1,0].plot(epoches_x_label, np.array([x[5] for x in data]), label='fake_score')

  axes[1,0].legend()
  axes[1,0].grid(True)
  # Plot 4 --------------------------------------------------------------

  axes[1,1].set_title('Validation accurency')
  axes[1,1].set_xlabel('Epochs')
  axes[1,1].set_ylabel('Accurency (%)')

  axes[1,1].plot(epoches_x_label, np.array([x[6] * 100 for x in data]), label='acc_real')
  axes[1,1].plot(epoches_x_label, np.array([x[7] * 100 for x in data]), label='acc_fake')
  axes[1,1].legend()
  axes[1,1].grid(True)

  plt.tight_layout()
  plt.show()

  if save_folder:
    os.makedirs(save_folder, exist_ok=True)
    fig.savefig(save_folder + f"/plot.svg")

In [ ]:
show_plt(train_info, result_folder)

### Metryki

In [ ]:
!pip install lpips
!pip install flip-evaluator

from skimage.metrics import structural_similarity as ssim
import lpips
from flip_evaluator import evaluate
from scipy.spatial.distance import directed_hausdorff
import cv2
from torchvision.utils import make_grid

In [ ]:
def scale_array(array=np.ndarray, min=float, max=float) -> np.ndarray:
  return np.interp(array, (array.min(), array.max()), (min, max))

In [ ]:
def calc_hausdorff(orginal=np.ndarray, proccessed=np.ndarray) -> float:
  threshold_h, _ = cv2.threshold(cv2.cvtColor(orginal, cv2.COLOR_RGB2GRAY), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
  orginal_edges = cv2.Canny(orginal, threshold1=threshold_h * 0.3, threshold2=threshold_h)

  threshold_h, _ = cv2.threshold(cv2.cvtColor(proccessed, cv2.COLOR_RGB2GRAY), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
  proccessed_edges = cv2.Canny(proccessed, threshold1=threshold_h * 0.3, threshold2=threshold_h)

  orginal_points_edges = np.column_stack(np.where(orginal_edges > 0))
  proccessed_points_edges = np.column_stack(np.where(proccessed_edges > 0))

  return max(directed_hausdorff(orginal_points_edges, proccessed_points_edges)[0], directed_hausdorff(proccessed_points_edges, orginal_points_edges)[0])

In [ ]:
def calc_metrics(test_loader, gen, device, lpips_net='squeeze', test_folder=None, result_folder=None, show_img=False):
  from google.colab.patches import cv2_imshow
  print(f"Urządzenie: {DEVICE}")

  gen.eval()

  loss_lpips = lpips.LPIPS(net=lpips_net).to(device)

  # metryki
  ssim_metric      = []
  lpips_metric     = []
  flip_metric      = []
  hausdorff_metric = []

  mertric_str = ""

  i = 0
  min = 0
  max = 1
  dynamicRange = "LDR"
  with torch.no_grad():
      for inputs, targets in test_loader:
          inputs = inputs.to(device)
          targets = targets.to(device)
          batch_size = inputs.shape[0]
          fake_images = gen(None, inputs)

          for fake, target in zip(fake_images, targets):
            print(f"\ttarget: min={target.min().item()}, max={target.max().item()}")
            print(f"\tfake: min={fake.min().item()}, max={fake.max().item()}")

            orginal = scale_array(target.cpu().numpy().transpose(1,2,0), min, max)
            processed =  scale_array(fake.cpu().numpy().transpose(1,2,0), min, max)

            print(f"\torginal: min={orginal.min().item()}, max={orginal.max().item()}")
            print(f"\tprocessed: min={processed.min().item()}, max={processed.max().item()}")

            ## ------- SSIM ----------
            ssim_metric.append(ssim(orginal, processed, full=False, multichannel=True, channel_axis=-1, data_range=(max - min)))

            ## ------- LIPIS ----------
            lpips_metric.append(loss_lpips(target, fake).item())

            ## ------- FLIP ----------
            flip_info = evaluate(orginal, processed, dynamicRange)
            flip_metric.append(flip_info[1])

            ## ------- HAUSDORFF  ----------
            orginal_255 = (orginal * 255).astype(np.uint8)
            processed_255 = (processed * 255).astype(np.uint8)

            hausdorff = calc_hausdorff(orginal_255, processed_255)

            if np.isfinite(hausdorff) and hausdorff > 0:
              hausdorff_metric.append(hausdorff)

            ## ------- RAPORT ----------

            line = f"Obraz i={i+1}, ssim={ssim_metric[-1]}, lpips={lpips_metric[-1]}, flip={flip_metric[-1]}, hausdorff={hausdorff}\n"
            mertric_str += line

            print(line)

            if show_img:
              cv2_imshow(orginal_255)
              cv2_imshow(processed_255)

            if test_folder:
              os.makedirs(test_folder, exist_ok=True)
              result_images = make_grid([target, fake], nrow=2)
              save_image(result_images * 0.5 + 0.5, f"{test_folder}/test_{i+1}.png")

            i += 1

  avg_ssim = np.mean(ssim_metric)
  avg_lpips =  np.mean(lpips_metric)
  avg_flip = np.mean(flip_metric)
  avg_hausdorff = np.mean(hausdorff_metric)

  if result_folder:
    os.makedirs(result_folder, exist_ok=True)
    mertric_str += f"\n\n### METRYKI DLA ZBIORU TESTOWEGO ###\n\n\tssim={avg_ssim}, lpips={avg_lpips}, flip={avg_flip}, hausdorff={avg_hausdorff}"
    with open(f"{result_folder}/metryki.txt", "w") as f:
      f.write(mertric_str)

  return avg_ssim, avg_lpips, avg_flip, avg_hausdorff

In [ ]:
avg_ssim, avg_lpips, avg_flip, avg_hausdorff = calc_metrics(test_loader, gen, DEVICE, test_folder=test_folder, result_folder=result_folder)
print(f"Metryki dla zbioru testowego: ssim={avg_ssim}, lpips={avg_lpips}, flip={avg_flip}, hausdorff={avg_hausdorff}")